In [1]:
import itertools
from collections import OrderedDict
import re
import nltk
from nltk.corpus import brown, gutenberg
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import heapq
from tqdm import tqdm
import numpy as np
import math
import time

In [2]:
nltk.download('gutenberg')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def preprocess(samples):
    preprocess_result = []
    for sent in samples:
        sent = [w.lower() for w in sent]
        sent = [w for w in sent if w not in stop_w]
        sent = [w.replace('\n', ' ') for w in sent]
        sent = [w for w in sent if pattern.fullmatch(w)]
        if len(sent) > 5:
            preprocess_result.append(sent)
    return preprocess_result

In [4]:
samples = gutenberg.sents(gutenberg.fileids()[3])
pattern = re.compile("[A-Za-z]+")
stop_w = set(stopwords.words('english'))
corpus = []
corpus = preprocess(samples)

In [5]:
print("samples length:", len(samples))  # 문장의 수가 3만개임
print("corpus length:", len(corpus))  # 실제 문장의 수는 corpus에 들어가있음

samples length: 30103
corpus length: 25481


In [6]:
fre_dist = FreqDist()
for sent in corpus:
    fre_dist.update(sent)
fre_dist = {k : v for k, v in fre_dist.items() if v > 5}

In [7]:
vocab_size = len(fre_dist)
idx_to_word = {idx: word for idx,  word in enumerate(fre_dist.keys())}
word_to_idx = {word: idx for idx, word in idx_to_word.items()}

In [8]:
corpus_indexed = [[word_to_idx[word] for word in sent if word in word_to_idx]for sent in corpus]
corpus_indexed = [sent for sent in corpus_indexed if len(sent) > 5]
fre_dist_indexed = {word_to_idx[w]: f for w, f in fre_dist.items()}

In [9]:
print(corpus_indexed)

[[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 5, 6, 11], [9, 12, 13, 14, 9, 15, 13, 4], [9, 16, 13, 17, 4, 16, 18], [9, 19, 20, 21, 22, 11, 20, 23, 11, 11], [9, 24, 21, 15, 11, 21, 11, 21], [9, 19, 20, 11, 25, 26, 27, 28, 29, 30, 20, 31, 32, 33], [9, 16, 31, 32, 0, 34, 27, 11, 16, 35, 9, 12, 14], [9, 19, 20, 0, 36, 37, 38, 39, 40, 41, 42, 43, 40, 42, 44, 45, 41, 5, 0], [0, 46, 37, 38, 39, 40, 41, 44, 43, 40, 42, 45, 41, 44, 9, 12, 14], [9, 19, 20, 47, 21, 25, 23, 17, 18, 20, 48, 49, 50, 51, 20, 47, 21, 25, 52, 13, 5, 0], [9, 24, 53, 54, 47, 55, 13, 56, 17, 13, 56, 18, 24, 57, 58], [9, 59, 21, 25, 52, 13, 5, 0, 56, 17, 18, 23, 13, 4, 9, 12, 14], [9, 19, 20, 11, 36, 37, 60, 61, 62, 63, 64, 65, 66, 0, 67, 21, 25], [9, 68, 54, 69, 70, 61, 71, 11, 46, 37, 60, 44, 69, 64, 44, 9, 12, 14], [9, 72, 73, 74, 75, 76, 11, 35, 20, 64, 75, 0], [9, 19, 20, 0, 36, 37, 70, 61, 44, 77, 78, 79, 80, 0, 44], [9, 24, 80, 0, 44, 77, 44, 69, 79, 81, 5, 0, 44, 9, 12, 14], [9, 19, 20, 82, 83, 84, 85, 86, 20, 87, 88, 89,

Huffman tree

In [10]:
class HeapNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
        self.index = None
        self.vector = None

    def __lt__(self, other):
        if other is None:
            return -1
        if not isinstance(other, HeapNode):
            return -1
        return self.freq < other.freq


class HuffmanCoding:
    def __init__(self):
        self.heap = []
        self.codes = {}
        self.reverse_mapping = {}
        self.merged_nodes = None

    def make_heap(self, frequency):  # frequency has a shape of  { word : frequency }
        for key in frequency:  # make a node, then push in list of heap queue
            node = HeapNode(key, frequency[key])
            heapq.heappush(self.heap, node)

    def merge_nodes(self):      # make nodes from low to high frequency and merge to tree.
        index = 0
        merged = None
        while len(self.heap) > 1:
            node1 = heapq.heappop(self.heap)
            node2 = heapq.heappop(self.heap)

            merged = HeapNode(None, node1.freq + node2.freq)
            merged.left = node1
            merged.right = node2
            merged.index = index                # index is reversed, i.e. root node has a biggest index.
            heapq.heappush(self.heap, merged)

            index += 1

        return merged

    def make_codes_helper(self, root, current_code):
        if root is None:
            return

        if root.char is not None:
            self.codes[root.char] = current_code
            self.reverse_mapping[current_code] = root.char
            return

        self.make_codes_helper(root.left, current_code + "0")
        self.make_codes_helper(root.right, current_code + "1")

    def make_codes(self):
        root = heapq.heappop(self.heap)
        current_code = ""
        self.make_codes_helper(root, current_code)

    def build(self, frequency):
        self.make_heap(frequency) # frequency를 기준으로 heapnode를 추가한다.
        merged = self.merge_nodes()
        self.make_codes()

        return self.codes, merged


def init_huffman_modified(fre_dist):
    h = HuffmanCoding()
    codes, merged = h.build(fre_dist)
    tree = {}
    max_depth = 0

    for word in tqdm(codes.keys(), desc='Building Huffman Tree', ncols=100):
        direction_code = codes[word]
        depth = len(direction_code)
        root = merged
        index_path = [root.index]
        direction_path = []
        for i in range(depth):
            direction_path.append(int(direction_code[i]))
            if direction_code[i] == '0':
                root = root.left
            else:
                root = root.right
            if root.index is not None:
                index_path.append(root.index)
        info = {'index_path': index_path, 'direction_path': direction_path, 'depth': depth}
        tree[word_to_idx[word]] = info

        if depth > max_depth:
            max_depth = depth

    return tree, max_depth

In [30]:
tree, max_depth = init_huffman_modified(fre_dist)
total = sum([item[1] for item in fre_dist.items()])
print(total)

TypeError: init_huffman_modified() takes 0 positional arguments but 1 was given

In [12]:
print(tree[0]['direction_path'])

[1, 1, 1, 1, 1, 1, 0, 1, 0]


Train

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import tqdm

In [14]:
def cosine_similarity(v1, v2):
    m1 = np.linalg.norm(v1)
    m2 = np.linalg.norm(v2)
    return np.dot(v1, v2) / (m1 * m2)

In [15]:
def sigmoid(xs):
    ans = 1 / (1 + np.exp(-xs))
    top = 1 / (1 + math.exp(6))
    bottom = 1 / (1 + math.exp(-6))
    for i, num in enumerate(ans[0]):
        if num < top:
            ans[0, i] = 0
        elif num > bottom:
            ans[0, i] = 1
    return ans

In [31]:
def similar_word(emb):
    index_to_word = idx_to_word
    word_to_index = word_to_idx
    embedding_norm = np.linalg.norm(emb, axis=1)
    norm_emb = emb / embedding_norm[:, None]
    word1 = word_to_index['king']
    word2 = word_to_index['queen']
    word3 = word_to_index['husband']
    answer = word_to_index['wife']

    target = norm_emb[word2] - norm_emb[word1] + norm_emb[word3]
    target = target / np.linalg.norm(target)

    max_index = answer
    max_sim = cosine_similarity(target, norm_emb[answer])
    for i in tqdm(range(len(word_to_index)), desc="Finding closest word to queen-king+husband", ncols=70):
        if i == word1 or i == word2 or i == word3 or i == answer:
            pass
        else:
            sim = cosine_similarity(norm_emb[i], target)
            if sim > max_sim:
                max_sim = sim
                max_index = i
    print(index_to_word[max_index])

In [ ]:
def train(cbow=True, ns=False, epochs=3, subsampling_t=1e-5, window_size=10, update_size=12):
    np.random.seed(1128)
    hidden_size = hidden_size
    neg_num = num_neg_samples

    model_type = 'ns' if ns else 'hs'
    train_type = 'cbow' if cbow else 'sg'
    emb_save_path = './results/embedding_{}_{}_{}_{}epoch.pkl'.format(model_type, train_type,
                                                                      subsampling_t, epochs)
    cont_save_path = './results/context_{}_{}_{}_{}epoch.pkl'.format(model_type, train_type,
                                                                     subsampling_t, epochs)
    node_mat_save_path = './results/node_mat_{}_{}_{}_{}epoch.pkl'.format(model_type, train_type,
                                                                          subsampling_t, epochs)

    if not os.path.isfile(cfg.freq_path):
        create_dictionary(cfg.train_files)
    frequency = pickle.load(open(cfg.freq_path, 'rb'))
    index_to_word = pickle.load(open(cfg.index_to_word_path, 'rb'))
    vocab_size = len(frequency)
    unigram_table = np.array(pickle.load(open(cfg.unigram_table_path, 'rb')))
    len_unigram_table = len(unigram_table)
    # tree, max_depth = pickle.load(open(cfg.tree_path, 'rb'))
    tree, max_depth = init_huffman_modified()
    total = sum([item[1] for item in frequency.items()])

    embedding = np.random.uniform(low=-0.5/300, high=0.5/300, size=(vocab_size, hidden_size)).astype('f')
    emb_grad_temp = []
    context = np.zeros_like(embedding).astype('f')  # for negative sampling

    node_mat = np.zeros((vocab_size-1, hidden_size)).astype('f')  # for hierarchical softmax
    node_mat_grad_temp = []

    starting_lr = 0.05 if cbow else 0.025
    min_loss = math.inf

    print("Start training on {} words".format(vocab_size))
    step = 0
    update_step = 0
    # logging_loss = 0
    start_time = time.time()
    lr = starting_lr

    update_size = update_size

    for epoch in range(epochs):
        data_paths = []
        total_pairs = 0
        print("======= Epoch {} training =======".format(epoch + 1))
        for i in range(len(cfg.train_files)):
            path = cfg.train_files[i]
            print("======= File number {} =======".format(i + 1))
            dataset = preprocess(path=path, frequency=frequency, total=total, window_size=window_size,
                                 cbow=cbow, subsampling_t=subsampling_t)
            data_path = "./preprocessed/data_{}_{}_{}.pkl".format(model_type, train_type, i)
            pickle.dump(dataset, open(data_path, 'wb'))
            data_paths.append(data_path)
            total_pairs += len(dataset)
        for i, data_path in enumerate(data_paths):
            print("======= File number {} =======".format(i + 1))
            dataset = pickle.load(open(data_path, 'rb'))
            loss = 0
            # lr = starting_lr * (1 - (epoch * 100 + i)/(epochs * 100))
            # if lr < starting_lr * 1e-4:
            #     lr = starting_lr * 1e-4
            print("Learning rate: {:.4f}".format(lr))

            file_count = 0
            lr_update_count = 0
            file_start_time = time.time()
            for input_idx, tgt_idx in tqdm(dataset, desc="Training", ncols=70):
                lr_update_count += 1
                if lr_update_count == 10000:
                    lr -= starting_lr * 10000 / (total_pairs * epochs)
                    if lr < starting_lr * 1e-4:
                        lr = starting_lr * 1e-4
                    lr_update_count = 0
                input_idx = np.array(input_idx)
                if cbow:
                    hidden = np.mean(embedding[input_idx], axis=0)  # (300, )
                else:  # sg
                    hidden = embedding[input_idx]   # (300, )
                hidden = hidden.reshape(1, 300)  # (1, 300)
                file_count += 1
                step += 1
                if ns:
                    while 1:
                        negs = np.random.randint(low=0, high=len_unigram_table, size=neg_num)
                        negs = unigram_table[negs]
                        if tgt_idx in negs:
                            continue
                        else:
                            break
                    targets = np.append(tgt_idx, negs)

                    ct = context[targets]  # (1 + neg_num, 300)
                    out = sigmoid(np.dot(hidden, ct.T))  # (1, 1 + neg_num)
                    p_loss = -np.log(out[:, :1] + 1e-7)
                    n_loss = -np.sum(np.log(1 - out[1:] + 1e-7))
                    loss += (p_loss.item() + n_loss.item())
                    # logging_loss += (p_loss.item() + n_loss.item())

                    out[:, :1] -= 1
                    context_grad = np.dot(out.T, hidden)
                    emb_grad = np.dot(out, context[targets])
                    if cbow:
                        emb_grad /= len(input_idx)
                    for j, target in enumerate(targets):
                        context[target] -= lr * context_grad[j]
                    # context[targets] -= lr * context_grad
                    embedding[input_idx] -= lr * emb_grad.squeeze()

                else:  # hs
                    # depth = tree[tgt_idx][-1]
                    # directions = tree[tgt_idx][:depth].reshape(1, -1)
                    # nodes = tree[tgt_idx][max_depth:(max_depth + depth)]
                    # depth = tree['depth']
                    info = tree[tgt_idx]
                    directions = info['direction_path']
                    nodes = info['index_path']
                    assert len(directions) == len(nodes)
                    out = sigmoid(np.dot(node_mat[nodes], hidden.T).T)  # (1, depth)
                    pair_loss = -np.log(np.prod(np.abs(directions - out)) + 1e-6)
                    loss += pair_loss
                    # logging_loss += pair_loss
                    dout = directions + out - 1  # (1, depth)
                    node_mat_grad = np.dot(dout.T, hidden)  # (depth, 300)
                    emb_grad = np.dot(dout, node_mat[nodes])  # (1, 300)
                    if cbow:
                        emb_grad /= len(input_idx)
                    node_mat_grad_temp.append((nodes, node_mat_grad))
                    emb_grad_temp.append((input_idx, emb_grad.squeeze()))
                    update_step += 1

                    if update_step == update_size or file_count == len(dataset):
                        for nodes, node_mat_grad in node_mat_grad_temp:
                            node_mat[nodes] -= lr * node_mat_grad
                        for input_indices, emb_grad in emb_grad_temp:
                            if cbow:
                                for idx in input_indices:
                                    embedding[idx] -= lr * emb_grad
                            else:
                                embedding[input_indices] -= lr * emb_grad

                        update_step = 0

                        node_mat_grad_temp.clear()
                        emb_grad_temp.clear()
                    else:
                        continue

                # if step % log_steps == 1:
                #     writer.add_scalar('Training loss', logging_loss/log_steps, int((step-1)/log_steps))
                #     logging_loss = 0
                # writer.flush()

            print("Number of pairs trained in this file: {}".format(file_count))
            print("Loss: {:.5f}".format(loss/file_count))
            print("Took {:.2f} hours for single file".format((time.time() - file_start_time)/3600))

            if loss < min_loss:
                min_loss = loss
                pickle.dump(embedding, open(emb_save_path, 'wb'))
                # if args.ns:
                #     pickle.dump(context, open(cont_save_path, 'wb'))
                # else:
                #     pickle.dump(node_mat, open(node_mat_save_path, 'wb'))
            similar_word(embedding)
        print("Training time: {:.2f} hours".format((time.time() - start_time) / 3600))

    if ns:
        return embedding, context
    else:
        return embedding, node_mat

In [26]:
hidden_size = 300
embedding = np.random.uniform(low=-0.5/300, high=0.5/300, size=(vocab_size, hidden_size)).astype('f')
emb_grad_temp = []
context = np.zeros_like(embedding).astype('f')  # for negative sampling
node_mat = np.zeros((vocab_size-1, hidden_size)).astype('f')  # for hierarchical softmax
node_mat_grad_temp = []

starting_lr = 0.05
min_loss = math.inf

In [27]:
print("Start training on {} words".format(vocab_size))
step = 0
update_step = 0
# logging_loss = 0
start_time = time.time()
lr = starting_lr
update_size = 12
epochs = 3
window_size = 10
subsampling_t = 1e-5
weight_decay = 1e-5

Start training on 4533 words


In [28]:
data_set = CBOWDataset(corpus_indexed)
data_loader = DataLoader(data_set, batch_size=100, num_workers=0)

In [29]:
embedding_dim = 50
net = CBOWHierarchicalSoftmax(vocab_size, hidden_size, fre_dist_indexed)
optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)

TypeError: 'module' object is not callable

In [ ]:
log_interval = 100
for epoch_i in range(10):
    total_loss = 0
    net.train()
    tk0 = tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0)
    for i, (context, center) in enumerate(tk0):

        loss = net(context, center)
        net.zero_grad()
        loss.backward()

        optimizer.step()

        total_loss += loss.item()
        if (i + 1) % log_interval == 0:
            tk0.set_postfix(loss=total_loss / log_interval)
            total_loss = 0